In [1]:
import os
from collections import Counter
from typing import *
import random
import json

In [2]:
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import KFold

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from torch.cuda.amp import autocast, GradScaler

from transformers import AutoConfig, AutoModel, AutoTokenizer, AdamW, get_constant_schedule_with_warmup, get_linear_schedule_with_warmup

In [3]:
from trainer import BasicTrainer
from model import DepParser
from utils import arc_rel_loss, uas_las

## Config

In [4]:
class CFG:
    data_file = '/root/diag_dep/data_new/1to200_0923.json'
    plm = 'hfl/chinese-electra-180g-large-discriminator'
    num_folds = 5
    trn_folds = [0, 1, 2, 3, 4]
    random_seed = 42
    num_epochs = 30
    batch_size = 4
    lr = 2e-5
    weight_decay = 0.01
    dropout = 0.2
    grad_clip = 1
    scheduler = 'linear'
    warmup_ratio = 0.12
    num_early_stop = 5
    max_length = 512
    num_labels = 40
    hidden_size = 400
    print_every = 1e9
    eval_every = 30
    cuda = True
    fp16 = True

## Seed and Device

In [5]:
def seed_everything(seed=CFG.random_seed):
    np.random.seed(seed%(2**32-1))
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic =True
    torch.backends.cudnn.benchmark = False

seed_everything()

In [6]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(f'Using device: {device}')

Using device: cuda


## Data

In [7]:
rel_dct = {
    'root': '根节点',
    'sasubj-obj': '同主同宾',
    'sasubj': '同主语',
    'dfsubj': '不同主语',
    'subj': '主语',
    'subj-in': '内部主语',
    'obj': '宾语',
    'pred': '谓语',
    'att': '定语',
    'adv': '状语',
    'cmp': '补语',
    'coo': '并列',
    'pobj': '介宾',
    'iobj': '间宾',
    'de': '的',
    'adjct': '附加',
    'app': '称呼',
    'exp': '解释',
    'punc': '标点',
    'frag': '片段',
    'repet': '重复',
    # rst
    'attr': '归属',
    'bckg': '背景',
    'cause': '因果',
    'comp': '比较',
    'cond': '状况',
    'cont': '对比',
    'elbr': '阐述',
    'enbm': '目的',
    'eval': '评价',
    'expl': '解释-例证',
    'joint': '联合',
    'manner': '方式',
    'rstm': '重申',
    'temp': '时序',
    'tp-chg': '主题变更',
    'prob-sol': '问题-解决',
    'qst-ans': '疑问-回答',
    'stm-rsp': '陈述-回应',
    'req-proc': '需求-处理',
}

In [8]:
rst_lst = [
    'attr','bckg','cause','comp','cond','cont','elbr','enbm','eval','expl','joint',
    'manner','rstm','temp','tp-chg','prob-sol','qst-ans','stm-rsp','req-proc',
]

In [9]:
rel2id = {key:idx for idx, key in enumerate(rel_dct.keys())}
print(rel2id)

{'root': 0, 'sasubj-obj': 1, 'sasubj': 2, 'dfsubj': 3, 'subj': 4, 'subj-in': 5, 'obj': 6, 'pred': 7, 'att': 8, 'adv': 9, 'cmp': 10, 'coo': 11, 'pobj': 12, 'iobj': 13, 'de': 14, 'adjct': 15, 'app': 16, 'exp': 17, 'punc': 18, 'frag': 19, 'repet': 20, 'attr': 21, 'bckg': 22, 'cause': 23, 'comp': 24, 'cond': 25, 'cont': 26, 'elbr': 27, 'enbm': 28, 'eval': 29, 'expl': 30, 'joint': 31, 'manner': 32, 'rstm': 33, 'temp': 34, 'tp-chg': 35, 'prob-sol': 36, 'qst-ans': 37, 'stm-rsp': 38, 'req-proc': 39}


In [10]:
tokenizer = AutoTokenizer.from_pretrained(CFG.plm)
print(len(tokenizer))
 
num_added_toks = tokenizer.add_tokens(['[root]', '[qst]', '[aws]'], special_tokens=True)
tokenizer.root_token = '[root]'
tokenizer.root_token_ids = tokenizer('[root]')['input_ids'][1]
print(f"add token: {tokenizer.root_token} {tokenizer.root_token_ids}")

tokenizer.root_token = '[qst]'
tokenizer.root_token_ids = tokenizer('[qst]')['input_ids'][1]
print(f"add token: {tokenizer.root_token} {tokenizer.root_token_ids}")

tokenizer.root_token = '[ans]'
tokenizer.root_token_ids = tokenizer('[ans]')['input_ids'][1]
print(f"add token: {tokenizer.root_token} {tokenizer.root_token_ids}")
print(len(tokenizer))

CFG.tokenizer = tokenizer

21128
add token: [root] 21128
add token: [qst] 21129
add token: [ans] 138
21131


In [11]:
with open(CFG.data_file, 'r', encoding='utf-8') as f:
    data = json.load(f)[:200]  # have annotated 200 data

sent_lst = []
max_len, trun_cnt = 0, 0
for d in data:
    word_lst = []
    for item in d['dialog']:
        word_lst.extend(item['utterance'].split(' '))
    if len(word_lst) > max_len:
        max_len = len(word_lst)
    if len(word_lst) > 450:
        trun_cnt += 1
    sent_lst.append(len(word_lst))

print(trun_cnt)
print(max_len)
print(np.mean(sent_lst))

1
626
209.725


In [12]:
class Dependency():
    def __init__(self, idx, word, head, rel):
        self.id = idx
        self.word = word
        self.tag = '_'
        self.head = head
        self.rel = rel

    def __str__(self):
        # example:  1	上海	_	NR	NR	_	2	nn	_	_
        values = [str(self.idx), self.word, "_", self.tag, "_", "_", str(self.head), self.rel, "_", "_"]
        return '\t'.join(values)

    def __repr__(self):
        return f"({self.id}, {self.word}, {self.tag}, {self.head}, {self.rel})"

In [17]:
def load_annoted(data_file):
    with open(CFG.data_file, 'r', encoding='utf-8') as f:
        data = json.load(f)[:200]  # have annotated 200 data
        
    sample_lst:List[List[Dependency]] = []
    
    for d in data:
        rel_dct = {}
        for tripple in d['relationship']:
            head, rel, tail = tripple
            head_uttr_idx, head_word_idx = [int(x) for x in head.split('-')]
            tail_uttr_idx, tail_word_idx = [int(x) for x in tail.split('-')]
            
            if rel == 'root' and head_uttr_idx != 0: # ignore root
                continue
                 
            if not rel_dct.get(tail_uttr_idx, None):
                rel_dct[tail_uttr_idx] = {tail_word_idx: [head, rel]}
            else:
                rel_dct[tail_uttr_idx][tail_word_idx] = [head, rel]
                
        sent_lens_accum = [0]
        for i, item in enumerate(d['dialog']):
            utterance = item['utterance']
            sent_lens_accum.append(sent_lens_accum[i] + len(utterance.split(' ')) + 1)
        
        dep_lst:List[Dependency] = []
        role_lst:List[str] = []
        for item in d['dialog']:
            turn = item['turn']
            utterance = item['utterance']
            # dep_lst:List[Dependency] = [Dependency(0, '[root]', -1, '_')]
            
            role = '[ans]' if item['speaker'] == 'A' else '[qst]'
            dep_lst.append(Dependency(sent_lens_accum[turn], role, -1, '_'))  
            
            for word_idx, word in enumerate(utterance.split(' ')):
                tail2head = rel_dct.get(turn, {1: [f'{turn}-{word_idx}', 'adjct']})
                head, rel = tail2head.get(word_idx + 1, [f'{turn}-{word_idx}', 'adjct'])  # some word annoted missed, padded with last word and 'adjct'
                head_uttr_idx, head_word_idx = [int(x) for x in head.split('-')]
                
                # only parse cross-utterance
                if turn != head_uttr_idx:
                    dep_lst.append(Dependency(sent_lens_accum[turn] + word_idx + 1, word, sent_lens_accum[head_uttr_idx] + head_word_idx, rel))  # add with accumulated length
                else:
                    dep_lst.append(Dependency(sent_lens_accum[turn] + word_idx + 1, word, -1, '_')) 
                
                # only parse rst relation
                # if rel not in rst_lst:
                #     dep_lst.append(Dependency(sent_lens_accum[turn] + word_idx + 1, word, -1, '_')) 
                # else:
                #     dep_lst.append(Dependency(sent_lens_accum[turn] + word_idx + 1, word, sent_lens_accum[head_uttr_idx] + head_word_idx, rel))  # add with accumulated length
        
            role_lst.append(item['speaker'])        
        sample_lst.append(dep_lst)
        
    return sample_lst

In [18]:
data = load_annoted(CFG.data_file)
data[0]

[(0, [ans], _, -1, _),
 (1, 有, _, -1, _),
 (2, 什么, _, -1, _),
 (3, 问题, _, -1, _),
 (4, 我, _, -1, _),
 (5, 可以, _, -1, _),
 (6, 帮, _, -1, _),
 (7, 您, _, -1, _),
 (8, 处理, _, -1, _),
 (9, 或, _, -1, _),
 (10, 解决, _, -1, _),
 (11, 呢, _, -1, _),
 (12, ?, _, -1, _),
 (13, [qst], _, -1, _),
 (14, 你, _, -1, _),
 (15, 好, _, 1, qst-ans),
 (16, [qst], _, -1, _),
 (17, 以前, _, -1, _),
 (18, 的, _, -1, _),
 (19, 手机, _, -1, _),
 (20, 号码, _, -1, _),
 (21, 销, _, 15, elbr),
 (22, 号, _, -1, _),
 (23, 了, _, -1, _),
 (24, ，, _, -1, _),
 (25, 密码, _, -1, _),
 (26, 也, _, -1, _),
 (27, 忘, _, -1, _),
 (28, 了, _, -1, _),
 (29, [qst], _, -1, _),
 (30, 改, _, 27, bckg),
 (31, 密码, _, -1, _),
 (32, 需要, _, -1, _),
 (33, 手机, _, -1, _),
 (34, 验证码, _, -1, _),
 (35, [ans], _, -1, _),
 (36, 还, _, -1, _),
 (37, 记得, _, 30, stm-rsp),
 (38, 您, _, -1, _),
 (39, 以前, _, -1, _),
 (40, 的, _, -1, _),
 (41, 手机, _, -1, _),
 (42, 号码, _, -1, _),
 (43, 吗, _, -1, _),
 (44, [qst], _, -1, _),
 (45, [, _, -1, _),
 (46, 数字, _, 37, qst-ans),
 (47

In [19]:
class DialogDataset(Dataset):
    def __init__(self, cfg):
        self.cfg = cfg
        self.inputs, self.offsets, self.heads, self.rels, self.masks = self.read_data()
        
    def read_data(self):
        inputs, offsets = [], []
        tags, heads, rels, masks = [], [], [], []
        
        for idx, deps in enumerate(load_annoted(self.cfg.data_file)):
            seq_len = len(deps)

            word_lst = [] 
            head_tokens = np.zeros(self.cfg.max_length, dtype=np.int64)  # same as root index is 0, constrainting by mask 
            rel_tokens = np.zeros(self.cfg.max_length, dtype=np.int64)
            mask_tokens = np.zeros(self.cfg.max_length, dtype=np.int64)
            for i, dep in enumerate(deps):
                if i == seq_len or i + 1== self.cfg.max_length:
                    break

                word_lst.append(dep.word)
                
                if int(dep.head) == -1 or int(dep.head) + 1 >= self.cfg.max_length:
                    head_tokens[i+1] = 0
                    mask_tokens[i+1] = 0
                else:
                    head_tokens[i+1] = int(dep.head)
                    mask_tokens[i+1] = 1
#                     head_tokens[i] = dep.head if dep.head != '_' else 0
                rel_tokens[i+1] = rel2id.get(dep.rel, 0)

            tokenized = tokenizer.encode_plus(word_lst, 
                                              padding='max_length', 
                                              truncation=True,
                                              max_length=self.cfg.max_length, 
                                              return_offsets_mapping=True, 
                                              return_tensors='pt',
                                              is_split_into_words=True)
            inputs.append({"input_ids": tokenized['input_ids'][0],
                          "token_type_ids": tokenized['token_type_ids'][0],
                           "attention_mask": tokenized['attention_mask'][0]
                          })

#                 sentence_word_idx = np.zeros(self.cfg.max_length, dtype=np.int64)
            sentence_word_idx = []
            for idx, (start, end) in enumerate(tokenized.offset_mapping[0][1:]):
                if start == 0 and end != 0:
                    sentence_word_idx.append(idx)
#                         sentence_word_idx[idx] = idx
            if len(sentence_word_idx) < self.cfg.max_length - 1:
                sentence_word_idx.extend([0]* (self.cfg.max_length - 1 - len(sentence_word_idx)))
            offsets.append(torch.as_tensor(sentence_word_idx))
#                 offsets.append(sentence_word_idx)

            heads.append(head_tokens)
            rels.append(rel_tokens)
            masks.append(mask_tokens)
                    
        return inputs, offsets, heads, rels, masks

    def __getitem__(self, idx):
        return self.inputs[idx], self.offsets[idx], self.heads[idx], self.rels[idx], self.masks[idx]
    
    def __len__(self):
        return len(self.rels)

In [20]:
dataset = DialogDataset(CFG)

## Tranining

In [21]:
kfold = KFold(n_splits=CFG.num_folds, shuffle=True, random_state=CFG.random_seed)

In [ ]:
for fold, (train_ids, val_ids) in enumerate(kfold.split(dataset)):
    if fold not in CFG.trn_folds:
        continue
    print(f'FOLD {fold}')
    print(f'{len(train_ids)}/{len(val_ids)}')
    print('--------------------------------')

    if CFG.cuda and torch.cuda.is_available:
        torch.cuda.empty_cache()

    random.shuffle(train_ids)
    random.shuffle(val_ids)

    tr_dataset = Subset(dataset, train_ids)
    va_dataset = Subset(dataset, val_ids)
    
    tr_iter = DataLoader(tr_dataset, batch_size=CFG.batch_size)
    va_iter = DataLoader(va_dataset, batch_size=CFG.batch_size)
    
    model = DepParser(CFG)
    
    optim = AdamW(model.parameters(), 
                      lr=CFG.lr,
                      weight_decay=CFG.weight_decay
                      )

    training_step = int(CFG.num_epochs * (len(train_ids) / CFG.batch_size))
    warmup_step = int(CFG.warmup_ratio * training_step)  
    lr_scheduler = get_linear_schedule_with_warmup(optimizer=optim, 
                                                        num_warmup_steps=warmup_step, 
                                                        num_training_steps=training_step)

    trainer = BasicTrainer(optim=optim, 
                        lr_scheduler=lr_scheduler,
                        trainset_size=len(train_ids), 
                        loss_fn=arc_rel_loss, 
                        metrics_fn=uas_las, 
                        config=CFG)
    
    best_res, best_state_dict = trainer.train(model=model, train_iter=tr_iter, val_iter=va_iter)
    print(best_res)
    with open("/root/autodl-tmp/diag_dep/k-fold/res.txt", 'a+') as f:
        f.write(f'{fold}\t {str(best_res)}\n')
    
    torch.save(best_state_dict, f"/root/autodl-tmp/diag_dep/k-fold/{fold}/model.bin")

FOLD 0
160/40
--------------------------------


Some weights of the model checkpoint at hfl/chinese-electra-180g-large-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/root/miniconda3/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.Ad

--epoch 0, step 0, loss 12.420459747314453
  {'UAS': 0.0125, 'LAS': 0.0}
--Evaluation:
-loss: 9.237998008728027  UAS: 0.008326508477819662  LAS: 0.002053794829024187 

--Best Evaluation: 
-loss: 9.237998008728027  UAS: 0.008326508477819662  LAS: 0.002053794829024187 



  3%|▎         | 1/30 [00:11<05:19, 11.02s/it]

--Evaluation:
-loss: 7.5951433181762695  UAS: 0.014968235494453905  LAS: 0.003648139685141119 

--Best Evaluation: 
-loss: 7.5951433181762695  UAS: 0.014968235494453905  LAS: 0.003648139685141119 



...

--Evaluation:
-loss: 3.2534000873565674  UAS: 0.5920171161613659  LAS: 0.4868162715742893 

--Best Evaluation: 
-loss: 3.2534000873565674  UAS: 0.5920171161613659  LAS: 0.4868162715742893 



 50%|█████     | 15/30 [02:51<02:52, 11.50s/it]

--Evaluation:
-loss: 3.305962085723877  UAS: 0.6086582437228103  LAS: 0.49687698004728525 

--Best Evaluation: 
-loss: 3.305962085723877  UAS: 0.6086582437228103  LAS: 0.49687698004728525 

--Evaluation:
-loss: 3.3987081050872803  UAS: 0.6083432377376121  LAS: 0.5079892202591555 

--Best Evaluation: 
-loss: 3.3987081050872803  UAS: 0.6083432377376121  LAS: 0.5079892202591555 



 53%|█████▎    | 16/30 [03:02<02:39, 11.40s/it]

--Evaluation:
-loss: 3.2910892963409424  UAS: 0.6281974231241065  LAS: 0.5247692785294907 

--Best Evaluation: 
-loss: 3.2910892963409424  UAS: 0.6281974231241065  LAS: 0.5247692785294907 



In [ ]:
os.system('shutdown')